# Setup

## Parse trajectory

In [ ]:
import pathlib

import matplotlib.colors

from analyzer.io.trajectory_parser import parse_trajectory
from analyzer.data.trajectory_data import TrajectoryUnit

folder = pathlib.Path("demos/uni-directional")
trajectories = {}
for file in folder.glob("uo*.txt"):
    trajectories[file.name] = parse_trajectory(trajectory_file=file, frame_rate=16.0, default_unit=TrajectoryUnit.CENTIMETER)

## Define geometry

### Define geometry in code

In [ ]:
import pygeos
from analyzer.data.geometry import Geometry

geometry = Geometry(
    pygeos.polygons(
        [
            (2.8, -6.5),
            (2.8, -4),
            (1.8, -4),
            (1.8, 4),
            (2.8, 4),
            (2.8, 8),
            (-1, 8),
            (-1, 4),
            (0, 4),
            (0, -4),
            (-1, -4),
            (-1, -6.5)
        ]
    )
)

### Define geometry in file

The same format as for JuPedSim geometries can be used as well directly from files.

**Note:** Currently obstacles will not be parsed from the file!

In [ ]:
import pathlib
from analyzer.data.geometry import Geometry
from analyzer.io.geometry_parser import parse_geometry

# geometry = parse_geometry(pathlib.Path("demos/crossing/geometry.xml"))

## Define measurement areas and lines

In [ ]:
import pygeos

measurement_area = pygeos.polygons([(0, -2), (0, 0), (1.8, 0), (1.8, -2)])

measurement_line = pygeos.linestrings([(0, 0), (1.8, 0)])

## Plot trajectories, geometry, and measurement areas/lines

In [ ]:
import matplotlib.pyplot as plt

for name, trajectory in trajectories.items():
    fig = plt.figure(figsize=(15, 20))
    ax1 = fig.add_subplot(111, aspect="equal")
    ax1.set_title(name)

    # Plot geometry
    ax1.plot(*pygeos.to_shapely(geometry.walkable_area).exterior.xy, color="k")

    ## Plot measurement area
    ax1.plot(*pygeos.to_shapely(measurement_area).exterior.xy, color="g")
    ax1.fill(*pygeos.to_shapely(measurement_area).exterior.xy, color="g", alpha=0.3)

    ## Plot measurement lines
    ax1.plot(*pygeos.to_shapely(measurement_line).xy, color="b")

    # Plot trajectories
    for id, ped in trajectory.data.groupby("ID"):
        p = ax1.plot(ped["X"], ped["Y"], label=id, alpha=0.1, color="r")
        ax1.scatter(
            ped[ped.frame == ped.frame.max()]["X"],
            ped[ped.frame == ped.frame.max()]["Y"],
            c=p[-1].get_color(),
            marker="x",
        )
    plt.show()

# Method A

## Compute n-t and flow

In [ ]:
from analyzer.methods.flow_calculator import compute_n_t
from analyzer.methods.flow_calculator import compute_flow

from analyzer.methods.velocity_calculator import compute_individual_velocity

nts = {}
flows = {}

for name, trajectory in trajectories.items():
    individual_speed = compute_individual_velocity(trajectory.data, trajectory.frame_rate, 10)

    nt, crossing = compute_n_t(trajectory.data, measurement_line, trajectory.frame_rate)

    delta_t = int(10 * trajectory.frame_rate)
    flow = compute_flow(nt, crossing, individual_speed, delta_t, trajectory.frame_rate)

    flows[name] = flow
    nts[name] = nt


## Plot n-t diagram

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(7, 7))
ax1 = fig.add_subplot(111)

for name, nt in nts.items():
    ax1.plot(nt.index, nt["Cumulative pedestrians"], label=name)

ax1.legend()
plt.show()

## Plot fundamental diagram

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(7, 7))
ax1 = fig.add_subplot(111)

for name, flow in flows.items():
    ax1.scatter(flow["Flow rate(1/s)"]/(flow["Mean velocity(m/s)"]*pygeos.length(measurement_line)), flow["Mean velocity(m/s)"], label=name)


ax1.set_xlim(left=0, right=4)
ax1.set_ylim(bottom=0, top=2.5)

ax1.set_xlabel("rho / 1/m^2")
ax1.set_ylabel("v/ m/s")
ax1.grid()
ax1.legend()
plt.show()

# Method B

## Compute individual velocity and density while passing the measurement area

In [ ]:
from analyzer.methods.velocity_calculator import compute_passing_speed
from analyzer.methods.density_calculator import compute_passing_density, compute_classic_density
from analyzer.methods.method_utils import compute_frame_range_in_area

passing_densities = {}
passing_speeds = {}

for name, trajectory in trajectories.items():
    frames_in_area, passing_area = compute_frame_range_in_area(trajectory.data, measurement_line, 2)

    passing_speed = compute_passing_speed(frames_in_area, trajectory.frame_rate, 2.)

    density_per_frame = compute_classic_density(trajectory.data, passing_area)
    passing_density = compute_passing_density(density_per_frame, frames_in_area)

    passing_speeds[name] = passing_speed
    passing_densities[name] = passing_density

## Plot fundamental diagram

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(7, 7))
ax1 = fig.add_subplot(111)

for name in passing_speeds.keys():
    ax1.scatter(passing_densities[name]["density"], passing_speeds[name]["speed"], label=name)
ax1.set_xlim(left=0, right=4)
ax1.set_ylim(bottom=0, top=2.5)

ax1.set_xlabel("rho / 1/m^2")
ax1.set_ylabel("v/ m/s")
ax1.grid()
ax1.legend()
plt.show()

# Method C

## Compute density and velocity

In [ ]:
from analyzer.methods.density_calculator import compute_classic_density
from analyzer.methods.velocity_calculator import compute_mean_velocity_per_frame

classic_densities = {}
mean_area_speeds = {}

for name, trajectory in trajectories.items():
    mean_area_speed, individual_speed_area = compute_mean_velocity_per_frame(
        trajectory.data, measurement_area, trajectory.frame_rate, 5
    )

    classic_density = compute_classic_density(trajectory.data, measurement_area)

    classic_densities[name] = classic_density
    mean_area_speeds[name] = mean_area_speed


## Plot time series data

In [ ]:
fig, ax = plt.subplots(nrows=len(trajectories.values()), ncols=2, figsize=(20, 60))
row = 0

ax[row, 1].set_title("Velocity", size='xx-large')
ax[row, 0].set_title("Density", size='xx-large')

for name, trajectory in trajectories.items():
    ax[row, 0].annotate(name, xy=(0.5, 1), xytext=(-ax[row, 0].yaxis.labelpad - 5, 0),
                xycoords=ax[row, 0].yaxis.label, textcoords='offset points',
                size='xx-large', ha='right', va='center', rotation=90)

    ax[row, 0].plot(classic_densities[name].index, classic_densities[name]["classic density"], alpha=1)
    ax[row, 0].set_xlim(left=0)
    ax[row, 0].set_ylim(bottom=0, top=4)
    ax[row, 0].set_xlabel("frame")
    ax[row, 0].set_ylabel("rho / 1/m^2")
    ax[row, 0].grid()

    ax[row, 1].plot(mean_area_speeds[name].index, mean_area_speeds[name], alpha=1)
    ax[row, 1].set_xlim(left=0,)
    ax[row, 1].set_ylim(bottom=0, top=3)
    ax[row, 1].set_xlabel("frame")
    ax[row, 1].set_ylabel("v / m/s")
    ax[row, 1].grid()

    row += 1


## Plot fundamental diagram

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(7, 7))
ax1 = fig.add_subplot(111)

ax1.set_title("Fundamental Diagram")

for name in classic_densities.keys():
    ax1.scatter(
        classic_densities[name]["classic density"],
        mean_area_speeds[name],
        alpha=1,
        marker="x",
        label = name,
    )
ax1.set_xlim(left=0, right=4)
ax1.set_ylim(bottom=0, top=2.5)

ax1.set_xlabel("rho / 1/m^2")
ax1.set_ylabel("v/ m/s")
ax1.grid()
ax1.legend()

# Method D

## Compute density and velocity

### Without cutoff radius

In [ ]:
from analyzer.methods.velocity_calculator import compute_voronoi_velocity
from analyzer.methods.density_calculator import compute_voronoi_density

voronoi_densities = {}
voronoi_velocities = {}
individual_velocities = {}
individuals = {}

for name, trajectory in trajectories.items():
    voronoi_density, individual = compute_voronoi_density(trajectory.data, measurement_area, geometry)

    voronoi_velocity, individual_velocity = compute_voronoi_velocity(
        trajectory.data, individual, trajectory.frame_rate, 5, measurement_area
    )

    voronoi_densities[name] = voronoi_density
    voronoi_velocities[name] = voronoi_velocity
    individual_velocities[name] = individual_velocity
    individuals[name] = individual

### With cutoff radius

In [ ]:
from analyzer.methods.velocity_calculator import compute_voronoi_velocity
from analyzer.methods.density_calculator import compute_voronoi_density

voronoi_densities_cutoff = {}
voronoi_velocities_cutoff = {}
individual_velocities_cutoff = {}
individual_cutoffs = {}
for name, trajectory in trajectories.items():

    voronoi_density_cutoff, individual_cutoff = compute_voronoi_density(
        trajectory.data, measurement_area, geometry, 0.8, 12
    )

    voronoi_velocity_cutoff, individual_velocity_cutoff = compute_voronoi_velocity(
        trajectory.data, individual_cutoff, trajectory.frame_rate, 5, measurement_area
    )

    voronoi_densities_cutoff[name] = voronoi_density_cutoff
    voronoi_velocities_cutoff[name] = voronoi_velocity_cutoff
    individual_velocities_cutoff[name] = individual_velocity_cutoff
    individual_cutoffs[name] = individual_cutoff

## Plot time-series data

## Plot fundamental diagram

In [ ]:
import matplotlib.pyplot as plt

fig, (ax0, ax1) = plt.subplots(nrows=1, ncols=2, figsize=(20, 6))
fig.suptitle("Fundamental diagram")

ax0.set_title("without cutoff")
for name in voronoi_densities.keys():
    ax0.scatter(
        voronoi_densities[name]["voronoi density"],
        voronoi_velocities[name],
        alpha=1,
        marker="x",
        label = name,
    )
ax0.set_xlim(left=0, right=4)
ax0.set_ylim(bottom=0, top=2.5)

ax0.set_xlabel("rho / 1/m^2")
ax0.set_ylabel("v/ m/s")
ax0.grid()
ax0.legend()

ax1.set_title("with cutoff")
for name in voronoi_densities_cutoff.keys():
    ax1.scatter(
        voronoi_densities_cutoff[name]["voronoi density"],
        voronoi_velocities_cutoff[name],
        alpha=1,
        marker="x",
        label = name,
    )
ax1.set_xlim(left=0, right=4)
ax1.set_ylim(bottom=0, top=2.5)

ax1.set_xlabel("rho / 1/m^2")
ax1.set_ylabel("v/ m/s")
ax1.grid()
ax1.legend()
plt.show()

# Profiles

## Compute velocity and density profiles

In [ ]:
import pandas as pd

from analyzer.methods.profile_calculator import compute_profiles
from analyzer.methods.velocity_calculator import compute_voronoi_velocity
from analyzer.methods.density_calculator import compute_voronoi_density

trajectory = list(trajectories.values())[7]

frames_data = trajectory.data[trajectory.data.frame.between(750, 1500)]

voronoi_density_frames, individual_frames = compute_voronoi_density(
    frames_data, measurement_area, geometry, 0.89, 12
)
voronoi_velocity_frames, individual_velocity_frames = compute_voronoi_velocity(
    trajectory.data, individual_frames, trajectory.frame_rate, 5, measurement_area
)

density_profiles, velocity_profiles = compute_profiles(
    pd.merge(individual_frames, individual_velocity_frames, on=["ID", "frame"], how="left"),
    geometry.walkable_area,
    0.25,
)


## Plot profiles

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

bounds = pygeos.bounds(geometry.walkable_area)

fig, (ax0, ax1) = plt.subplots(nrows=1, ncols=2, figsize=(10, 10))

ax0.set_title("Density")
cm = ax0.imshow(
    np.transpose(np.mean(density_profiles, axis=0)),
    extent=[bounds[0], bounds[2], bounds[1], bounds[3]],
    interpolation="None",
    cmap="RdBu_r",
    vmin=0,
    vmax=5,
)
fig.colorbar(cm, ax=ax0, shrink=0.3)
ax0.plot(*pygeos.to_shapely(geometry.walkable_area).exterior.xy, color="k")

ax1.set_title("Velocity")
cm = ax1.imshow(
    np.transpose(np.mean(velocity_profiles, axis=0)),
    extent=[bounds[0], bounds[2], bounds[1], bounds[3]],
    interpolation="None",
    cmap="RdBu",
    vmin=0,
    vmax=1.5,
)
fig.colorbar(cm, ax=ax1, shrink=0.3)

ax1.plot(*pygeos.to_shapely(geometry.walkable_area).exterior.xy, color="k")

fig.tight_layout()

# Plots

### Plot voronoi cells

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl

import pandas as pd
from enum import Enum

class ColorMode(Enum):
    Velocity = 1
    Density = 2

# Choose trajectory file to plot
name = list(trajectories.keys())[6]

# Choose frame to plot
frame = 700

# Choose which color mode used to display the voronoi cells. Can be Velocity or Density
color_mode = ColorMode.Velocity

# Choose which voronoi data to plot
use_cut_off_data = True

# merge all relevant data into one data frame
voronoi_data = individual_cutoffs[name] if use_cut_off_data else individuals[name]
velocity_data = individual_velocities_cutoff[name][["ID", "frame", "speed"]] if use_cut_off_data else individual_velocities[name][["ID", "frame", "speed"]]
all_data = pd.merge(trajectories[name].data, voronoi_data, on=["ID", "frame"]).merge(velocity_data, on=["ID", "frame"])
# data = all_data
data = all_data[all_data.frame == frame]

# Start plotting
fig = plt.figure(figsize=(15, 20))
fig.suptitle(name)
ax1 = fig.add_subplot(111, aspect="equal")

# Plot geometry
ax1.plot(*pygeos.to_shapely(geometry.walkable_area).exterior.xy, color="k")

# Plot measurement area
ax1.plot(*pygeos.to_shapely(measurement_area).exterior.xy, color="g")

# Create color mapping for velocity/density to color
my_cmap = plt.get_cmap("RdBu") if color_mode==ColorMode.Velocity else plt.get_cmap("RdBu_r")
norm = mpl.colors.Normalize(vmin=0, vmax=4 if color_mode==ColorMode.Velocity else 8)
scalar_mappable = mpl.cm.ScalarMappable(norm=norm, cmap=my_cmap)

# Plot voronoi cells
for id, ped in data.groupby("ID"):
    poly = pygeos.to_shapely(ped["individual voronoi"])[0]

    color = "gray"
    ax1.plot(*poly.exterior.xy, alpha=0.2, color=color)
    ax1.fill(*poly.exterior.xy, fc=color, alpha=0.1)

    sc = ax1.scatter(ped["X"], ped["Y"], c=color, alpha = 0.5)

    if not pygeos.is_empty(ped["intersection voronoi"].iat[0]):
        poly = pygeos.to_shapely(ped["intersection voronoi"])[0]
        color = scalar_mappable.to_rgba(ped['speed']) if color_mode==ColorMode.Velocity else scalar_mappable.to_rgba(1/poly.area)
        ax1.plot(*poly.exterior.xy, alpha=0.5, color=color)
        ax1.fill(*poly.exterior.xy, fc=color, alpha=0.4)

fig.colorbar(scalar_mappable, label="v / m/s" if color_mode==ColorMode.Velocity else "rho / 1/m^2")
plt.show()
